# This is a demo that shows how tables are combined to create a schema

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import datajoint as dj

your_config_file = '../djimaging/djconfig/djconf_template.json'  # WARNING: This will not work with the empty template

assert os.path.isfile(your_config_file), 'File not found'

# Load configuration for user
dj.config.load(your_config_file)

In [ ]:
assert "database.user" in dj.config, 'Set user name'

In [ ]:
dj.config['schema_name'] = f'ageuler_{dj.config["database.user"]}_test'

In [ ]:
assert "schema_name" in dj.config, 'Set schema name'

realusername = !whoami
realusername = realusername[0]

assert dj.config["database.user"] == realusername, 'How are you?'
assert dj.config["database.user"] in dj.config["schema_name"], 'Are you sure you want to connect to this DataBase'

In [ ]:
dj.conn()

In [ ]:
schema = dj.schema(dj.config["schema_name"], locals())
schema

In [ ]:
schema.drop()

In [ ]:
dj.conn()
schema = dj.schema(dj.config["schema_name"], locals())
schema

In [ ]:
from djimaging.core_tables import userinfo

@schema
class UserInfo(userinfo.UserInfo):
    pass

In [ ]:
dj.ERD(schema)

In [ ]:
UserInfo()

In [ ]:
from djimaging.core_tables import experiment

@schema
class Experiment(experiment.Experiment):
    userinfo_table = UserInfo

In [ ]:
erd = dj.ERD(schema)
erd

In [ ]:
from djimaging.core_tables import field

@schema
class Field(field.Field):
    userinfo_table = UserInfo
    experiment_table = Experiment

@schema
class Roi(field.Roi):
    field_table = Field

In [ ]:
dj.ERD(schema)

In [ ]:
from djimaging.core_tables import stimulus

@schema
class Stimulus(stimulus.Stimulus):
    pass

@schema
class Presentation(stimulus.Presentation):
    experiment_table = Experiment
    userinfo_table = UserInfo
    field_table = Field
    stimulus_table = Stimulus    

In [ ]:
dj.ERD(schema)

In [ ]:
from djimaging.core_tables import location

@schema
class RelativeFieldLocation(location.RelativeFieldLocation):
    field_table = Field
    expinfo_table = Experiment.ExpInfo

@schema
class RetinalFieldLocation(location.RetinalFieldLocation):
    expinfo_table = Experiment.ExpInfo

In [ ]:
dj.ERD(schema)

In [ ]:
from djimaging.core_tables import traces

@schema
class Traces(traces.Traces):
    presentation_table = Presentation
    field_table = Field
    roi_table = Roi

@schema
class DetrendParams(traces.DetrendParams):
    pass

@schema
class DetrendTraces(traces.DetrendTraces):
    presentation_table = Presentation
    detrendparams_table = DetrendParams
    traces_table = Traces

@schema
class DetrendSnippets(traces.DetrendSnippets):
    stimulus_table = Stimulus
    presentation_table = Presentation
    traces_table = Traces
    detrendtraces_table = DetrendTraces

@schema
class Averages(traces.Averages):
    detrendsnippets_table = DetrendSnippets

In [ ]:
dj.ERD(schema)

In [ ]:
Averages()